# `Heuristic.ipynb`: Calculate a heruistic for the travel time
We calculate the heuristic the follwing way:<br>
The heuristic from station a to station b is the minimum time one has to be sitting in a train to get from a to b. In other words, we assume that one can take trains like cars, no timetable limits are imposed. This will clearly give an overly optimistic estimate, as the waiting times from the timetables are not included.

In [1]:
import getpass
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import unix_timestamp, round
from pyspark.sql import functions as F

spark = SparkSession \
    .builder \
    .master("yarn") \
    .appName('sbb-{0}'.format(getpass.getuser())) \
    .config('spark.executor.memory', '4g') \
    .config('spark.executor.instances', '5') \
    .config('spark.port.maxRetries', '100') \
    .getOrCreate()
spark

In [2]:
import numpy as np
import pandas as pd

### Distance Filter
First, we filter on only the stations in a 10km radius around Zürich. This allows for computation in "reasonable" time on the cluster.

In [3]:
def distance_squared(n1,e1, n2, e2):
    '''Calculates the euclidean distance between two points'''
    eucl_dist2 = ((n1-n2)*(n1-n2)+ (e1-e2)*(e1-e2))
    return eucl_dist2

coords_zurich = (683144.0, 248040.0) # X, Y  (E,N)

In [4]:
# Get the data of train station localisation
df_stops = spark.read.csv('stops.txt', sep=';', header=True).select('Dst-Bezeichnung-offiziell','KOORDE','KOORDN')\
.withColumnRenamed('Dst-Bezeichnung-offiziell','station_name')

In [5]:
df_stops = df_stops.withColumn('dist2', distance_squared(coords_zurich[1], coords_zurich[0], df_stops.KOORDN, df_stops.KOORDE))

In [6]:
df_stops = df_stops.filter(df_stops.dist2<=10000**2).persist()

In [7]:
stops_filter = df_stops.select('station_name')

In [8]:
swiss_data = spark.read.csv('/datasets/sbb/2017/10/2017-10-17istdaten.csv.bz2', header=True, sep=";").cache()

In [9]:
istdaten = swiss_data.join(stops_filter, swiss_data.HALTESTELLEN_NAME == stops_filter.station_name, 'inner').persist()

In [10]:
df = istdaten.select('FAHRT_BEZEICHNER', 'HALTESTELLEN_NAME', 'ANKUNFTSZEIT', 'ABFAHRTSZEIT').persist()

In [11]:
df = df.filter(df.ANKUNFTSZEIT.isNotNull() & df.ABFAHRTSZEIT.isNotNull())

In [12]:
df.show()

+----------------+-----------------+----------------+----------------+
|FAHRT_BEZEICHNER|HALTESTELLEN_NAME|    ANKUNFTSZEIT|    ABFAHRTSZEIT|
+----------------+-----------------+----------------+----------------+
|  85:11:1255:001|        Zürich HB|17.10.2017 08:26|17.10.2017 08:37|
|  85:11:1258:001|        Zürich HB|17.10.2017 21:23|17.10.2017 21:34|
|  85:11:1507:002|        Zürich HB|17.10.2017 06:30|17.10.2017 06:39|
|  85:11:1507:002| Zürich Flughafen|17.10.2017 06:49|17.10.2017 06:51|
|  85:11:1509:002|        Zürich HB|17.10.2017 07:30|17.10.2017 07:39|
|  85:11:1509:002| Zürich Flughafen|17.10.2017 07:49|17.10.2017 07:51|
|  85:11:1510:002| Zürich Flughafen|17.10.2017 07:11|17.10.2017 07:13|
|  85:11:1510:002|        Zürich HB|17.10.2017 07:23|17.10.2017 07:30|
|  85:11:1511:002|        Zürich HB|17.10.2017 08:30|17.10.2017 08:39|
|  85:11:1511:002| Zürich Flughafen|17.10.2017 08:49|17.10.2017 08:51|
|  85:11:1512:002| Zürich Flughafen|17.10.2017 08:11|17.10.2017 08:13|
|  85:

### Self-Join to get trips
We perform a join with itself, to get the trips. The time for each trip is negative if the trip goes in the opposite direction.

In [13]:
columns = ['FAHRT_BEZEICHNER', 'HALTESTELLEN_NAME', 'ANKUNFTSZEIT', 'ABFAHRTSZEIT']
df1 = df
df2 = df
for c in columns:
    df1 = df1.withColumnRenamed(c, c+'_1')
    df2 = df2.withColumnRenamed(c, c+'_2')

In [14]:
df1.show(2)
df2.show(2)

+------------------+-------------------+----------------+----------------+
|FAHRT_BEZEICHNER_1|HALTESTELLEN_NAME_1|  ANKUNFTSZEIT_1|  ABFAHRTSZEIT_1|
+------------------+-------------------+----------------+----------------+
|    85:11:1255:001|          Zürich HB|17.10.2017 08:26|17.10.2017 08:37|
|    85:11:1258:001|          Zürich HB|17.10.2017 21:23|17.10.2017 21:34|
+------------------+-------------------+----------------+----------------+
only showing top 2 rows

+------------------+-------------------+----------------+----------------+
|FAHRT_BEZEICHNER_2|HALTESTELLEN_NAME_2|  ANKUNFTSZEIT_2|  ABFAHRTSZEIT_2|
+------------------+-------------------+----------------+----------------+
|    85:11:1255:001|          Zürich HB|17.10.2017 08:26|17.10.2017 08:37|
|    85:11:1258:001|          Zürich HB|17.10.2017 21:23|17.10.2017 21:34|
+------------------+-------------------+----------------+----------------+
only showing top 2 rows



In [15]:
full_df = df1.join(df2, df1.FAHRT_BEZEICHNER_1==df2.FAHRT_BEZEICHNER_2)

In [16]:
times_df = full_df.withColumn('time', (unix_timestamp("ANKUNFTSZEIT_1", 'dd.MM.yyyy HH:mm') - \
                               unix_timestamp("ABFAHRTSZEIT_2",'dd.MM.yyyy HH:mm'))/60)\
                                .select('HALTESTELLEN_NAME_1', 'HALTESTELLEN_NAME_2', 'time')
times_df = times_df.filter(times_df.time>=0)

In [17]:
times_df.show()

+-------------------+-------------------+----+
|HALTESTELLEN_NAME_1|HALTESTELLEN_NAME_2|time|
+-------------------+-------------------+----+
|   Zürich Flughafen|          Zürich HB|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zürich HB|   Zürich Flughafen|10.0|
|   Zürich Flughafen|          Zürich HB|10.0|
|          Zü

After calculating all the travel times, we now build our precomputed dictionnary and Distance Matrix. We initialise the Distance matrix with 12h, and add the connections.

In [18]:
stations = [r[0] for r in times_df.select('HALTESTELLEN_NAME_1').distinct().collect()]

In [19]:
n_stations = len(stations)

In [20]:
min_travel_times = np.full((n_stations,n_stations), 60*24.) #maximal travel time is 1 day (this is way to much)

In [21]:
min_travel_times

array([[1440., 1440., 1440., ..., 1440., 1440., 1440.],
       [1440., 1440., 1440., ..., 1440., 1440., 1440.],
       [1440., 1440., 1440., ..., 1440., 1440., 1440.],
       ...,
       [1440., 1440., 1440., ..., 1440., 1440., 1440.],
       [1440., 1440., 1440., ..., 1440., 1440., 1440.],
       [1440., 1440., 1440., ..., 1440., 1440., 1440.]])

In [22]:
trips_df = times_df.filter(times_df.HALTESTELLEN_NAME_1!=times_df.HALTESTELLEN_NAME_2)

In [23]:
dist_times = trips_df.groupBy('HALTESTELLEN_NAME_1', 'HALTESTELLEN_NAME_2')\
                        .agg(F.min(trips_df.time))

In [24]:
df_pd = dist_times.toPandas()

In [25]:
stations_dict = dict(zip(stations, range(n_stations)))

### Use Floyd-Warshall Algorithm to find shortest path distances
Now, we use the Floyd-Warshall Algorithm to calculate our distance matrix.

First, add the train connections to our graph.

In [26]:
# add edges
for i, r in df_pd.iterrows():
    _i = stations_dict[r['HALTESTELLEN_NAME_1']]
    _j = stations_dict[r['HALTESTELLEN_NAME_2']]
    _t_min = r['min(time)']
    min_travel_times[_j,_i] = _t_min

Second, self-distance is 0.

In [27]:
# self-distance is 0
for i in range(n_stations):
    min_travel_times[i,i]=0

Third, add walking times.

In [28]:
# first, load all the necessary data
import pickle
with open('./data/station_from_index.pkl', 'rb') as handle:
    station_from_index = pickle.load(handle)
with open('./data/index_from_station.pkl', 'rb') as handle:
    index_from_station = pickle.load(handle)
walking_distances = np.load('./data/walking_distances.npy')
walking_distances = np.sqrt(walking_distances)

# definition of the maximum walking time
MAX_WALKING_TIME = 5*60

In [29]:
for s1 in list(stations_dict.keys()):
    if s1 in index_from_station.keys():
        # get the distances for all stations
        walking_times = walking_distances[index_from_station[s1],:]
        # get stations in walking distance
        close_stations = np.argwhere(walking_times<MAX_WALKING_TIME)

        # get the names of the stations
        station_names = [station_from_index[i] for i in close_stations.flatten().tolist()  \
                         if i!=index_from_station[s1]]
        # get the estimated arrival times for all stations
        arrival_times = (walking_times[close_stations].flatten()).tolist()
        for i in range(len(station_names)):
            s2 = station_names[i]
            if s2 not in stations_dict.keys():
                continue
            _i = stations_dict[s2]
            _j = stations_dict[s1]
            _t_min = arrival_times[i]/60
            if _t_min<min_travel_times[_j,_i]:
                min_travel_times[_j,_i] = _t_min

Now, find distances on this graph.

In [30]:
# main path finding algorithm
for k in range(n_stations):
    for i in range(n_stations):
        for j in range(n_stations):
            if min_travel_times[i,j] > min_travel_times[i,k] + min_travel_times[k,j]:
                min_travel_times[i,j] = min_travel_times[i,k] + min_travel_times[k,j]

In [31]:
min_travel_times

array([[ 0.        ,  9.        ,  9.        , ..., 10.        ,
         9.4472774 , 11.        ],
       [ 9.        ,  0.        ,  2.        , ..., 14.        ,
        11.68960536, 15.        ],
       [ 8.        ,  2.        ,  0.        , ..., 13.        ,
         9.68960536, 14.        ],
       ...,
       [12.        , 16.        , 16.        , ...,  0.        ,
        17.        ,  7.        ],
       [10.36895614, 13.        , 12.        , ..., 15.36895614,
         0.        , 16.36895614],
       [14.        , 18.        , 18.        , ...,  8.        ,
        19.        ,  0.        ]])

In [32]:
np.save('./data/min_travel_times', min_travel_times)

In [33]:
import pickle

with open('./data/stations_dict', 'wb') as handle:
    pickle.dump(stations_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### The function to use in the algorithm

Below, we give the function to be used in the algorithm.

In [34]:
import pickle
with open('./data/stations_dict', 'rb') as handle:
    stations_dict = pickle.load(handle)
min_travel_times = np.load('./data/min_travel_times.npy')

def get_heuristic(city_a,city_b):
    '''
    Calculates the heuristic of time in minutes between two city taking a train
    '''
    # try to find connection in our adjacency matrix calculated in Heuristic.ipynb
    if city_a in stations_dict and city_b in stations_dict:
        i = stations_dict[city_a]
        j = stations_dict[city_b]
        estimated_time = min_travel_times[i,j]
        if estimated_time!=1440:
            # estimated_time=1440: no possible connection was found when building the Heuristic adjacency matrix
            return estimated_time
    
    avg_speed = 1600
    try :
        df_a = df_stops.select('KOORDE','KOORDN').where(df_stops['station_name'] ==city_a).collect()
        x_a = float(df_a[0]['KOORDE'])
        y_a = float(df_a[0]['KOORDN'])
    except IndexError :
        print('Name ',city_a,'is invalide')
        return 0
    try :
        df_b = df_stops.select('KOORDE','KOORDN').where(df_stops['station_name'] ==city_b).collect()
        x_b = float(df_b[0]['KOORDE'])
        y_b = float(df_b[0]['KOORDN'])
    except IndexError :
        print('Name ',city_b,'is invalide')
        return 0
    eucl_dist = math.sqrt((x_a-x_b)*(x_a-x_b)+ (y_a-y_b)*(y_a-y_b))
    return eucl_dist/avg_speed

In [35]:
get_heuristic('Zürich HB','Zürich, Bahnhofquai/HB')

1.0